In [1]:
# Local Feature Stencil Code
# Written by James Hays for CS 143 @ Brown / CS 4476/6476 @ Georgia Tech with Henry Hu <henryhu@gatech.edu>
# Edited by James Tompkin
# Adapted for python by asabel and jdemari1 (2019)

import csv
import sys
import argparse
import numpy as np

import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

from skimage import io, filters, feature, img_as_float32
from skimage.transform import rescale
from skimage.color import rgb2gray

from utils import *
import cv2
import student
import visualize
from helpers import cheat_interest_points, evaluate_correspondence

In [2]:
# This script
# (1) Loads and resizes images
# (2) Finds interest points in those images                 (you code this)
# (3) Describes each interest point with a local feature    (you code this)
# (4) Finds matching features                               (you code this)
# (5) Visualizes the matches
# (6) Evaluates the matches based on ground truth correspondences

def load_data(file_name):
    """
    1) Load stuff
    There are numerous other image sets in the supplementary data on the
    project web page. You can simply download images off the Internet, as
    well. However, the evaluation function at the bottom of this script will
    only work for three particular image pairs (unless you add ground truth
    annotations for other image pairs). It is suggested that you only work
    with the two Notre Dame images until you are satisfied with your
    implementation and ready to test on additional images. A single scale
    pipeline works fine for these two images (and will give you full credit
    for this project), but you will need local features at multiple scales to
    handle harder cases.

    If you want to add new images to test, create a new elif of the same format as those
    for notre_dame, mt_rushmore, etc. You do not need to set the eval_file variable unless
    you hand create a ground truth annotations. To run with your new images use
    python main.py -p <your file name>.

    :param file_name: string for which image pair to compute correspondence for

        The first three strings can be used as shortcuts to the
        data files we give you

        1. notre_dame
        2. mt_rushmore
        3. e_gaudi

    :return: a tuple of the format (image1, image2, eval file)
    """

    # Note: these files default to notre dame, unless otherwise specified
    # image1_file = "../data/NotreDame/NotreDame1.jpg"
    # image2_file = "../data/NotreDame/NotreDame2.jpg"

    image1_file = "../data/NotreDameCopy/Notre1.jpg"
    image2_file = "../data/NotreDameCopy/Notre2.jpg"

    eval_file = "../data/NotreDame/NotreDameEval.mat"

    if file_name == "notre_dame":
        pass
    elif file_name == "mt_rushmore":
        image1_file = "../data/MountRushmore/Mount_Rushmore1.jpg"
        image2_file = "../data/MountRushmore/Mount_Rushmore2.jpg"
        eval_file = "../data/MountRushmore/MountRushmoreEval.mat"
    elif file_name == "e_gaudi":
        image1_file = "../data/EpiscopalGaudi/EGaudi_1.jpg"
        image2_file = "../data/EpiscopalGaudi/EGaudi_2.jpg"
        eval_file = "../data/EpiscopalGaudi/EGaudiEval.mat"

    image1 = cv2.imread(image1_file)
    image1 = image1.astype(np.float32)/255
    image1 = image1[:, :, ::-1]
    # image2 = img_as_float32(io.imread(image2_file))
    image2 = cv2.imread(image2_file)
    image2 = image2.astype(np.float32)/255
    image2 = image2[:, :, ::-1]

    return image1, image2, eval_file


In [4]:
# (1) Load in the data
image1, image2, eval_file = load_data("notre_dame")
# # width and height of each local feature, in pixels
feature_width = 16
scale_factor = 0.5

image1 = cv2.resize(image1, (0, 0), fx=scale_factor, fy=scale_factor)
image2 = cv2.resize(image2, (0, 0), fx=scale_factor, fy=scale_factor)
image1_bw = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
image2_bw = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)

In [ ]:
image_file = "../data/NotreDameCopy/Notre1.jpg"
image_test=cv2.imread(image_file)
Xderivative = cv2.Sobel(image_test, cv2.CV_64F,1,0,ksize=5)
Yderivative = cv2.Sobel(image_test, cv2.CV_64F,0,1,ksize=5)
plt.imshow(Yderivative)
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [5]:
# (2) Find distinctive points in each image. See Szeliski 4.1.1
# !!! You will need to implement get_interest_points. !!!

print("Getting interest points...")

# For development and debugging get_features and match_features, you will likely
# want to use the ta ground truth points, you can comment out the precedeing two
# lines and uncomment the following line to do this.

#(x1, y1, x2, y2) = cheat_interest_points(eval_file, scale_factor)
print("Getting interest points of image1...")
(x1, y1) = student.get_interest_points(image1_bw, feature_width)
print("Getting interest points of image2...")
(x2, y2) = student.get_interest_points(image2_bw, feature_width)

# if you want to view your corners uncomment these next lines!
# print("Show the interest points!")
# plt.imshow(image1, cmap="gray")
# plt.scatter(x1, y1, alpha=0.9, s=3)
# plt.show()

# plt.imshow(image2, cmap="gray")
# plt.scatter(x2, y2, alpha=0.9, s=3)
# plt.show()

print('{:d} corners in image 1, {:d} corners in image 2'.format(len(x1), len(x2)))
print("Done!")

Getting interest points...
Getting interest points of image1...
Getting interest points of image2...
3025 corners in image 1, 3025 corners in image 2
Done!


In [6]:
# 3) Create feature vectors at each interest point. Szeliski 4.1.2
# !!! You will need to implement get_features. !!!

print("Getting features...")
image1_features = student.get_features(image1_bw, x1, y1, feature_width)
image2_features = student.get_features(image2_bw, x2, y2, feature_width)
print("image1_features len:",len(image1_features))
print("Done!")

Getting features...
image1_features len: 3025
Done!


In [7]:
# 4) Match features. Szeliski 4.1.3
# !!! You will need to implement match_features !!!

print("Matching features...")
matches, confidences = student.match_features(image1_features, image2_features)
print('{:d} matches from {:d} corners'.format(len(matches), len(x1)))

Matching features...
164 matches from 3025 corners


In [8]:
if len(matches.shape) == 1:
    print( "No matches!")

print("Done!")
print("matche shape:",matches.shape)

Done!
matche shape: (164, 2)


In [8]:
# 5) Visualization

# You might want to do some preprocessing of your interest points and matches
# before visualizing (e.g. only visualizing 100 interest points). Once you
# start detecting hundreds of interest points, the visualization can become
# crowded. You may also want to threshold based on confidence

# visualize.show_correspondences produces a figure that shows your matches
# overlayed on the image pairs. evaluate_correspondence computes some statistics
# about the quality of your matches, then shows the same figure. If you want to
# just see the figure, you can uncomment the function call to visualize.show_correspondences


num_pts_to_visualize = matches.shape[0]
print("Matches: " + str(num_pts_to_visualize))
# visualize.show_correspondences(image1, image2, x1, y1, x2, y2, matches, filename=args.pair + "_matches.jpg")

## 6) Evaluation
# This evaluation function will only work for the Notre Dame, Episcopal
# Gaudi, and Mount Rushmore image pairs. Comment out this function if you
# are not testing on those image pairs. Only those pairs have ground truth
# available.
#
# It also only evaluates your top 100 matches by the confidences
# that you provide.
#
# Within evaluate_correspondences(), we sort your matches in descending order
#
num_pts_to_evaluate = matches.shape[0]

evaluate_correspondence(image1, image2, eval_file, scale_factor,
    x1, y1, x2, y2, matches, confidences, num_pts_to_visualize)

Matches: 164
105.0 total good matches, 44 total bad matches.
70.46979865771812% precision
68% accuracy (top 100)
Vizualizing...


In [7]:
image1_features.shape

(3025, 128)